# Stream Processing - Exercise 8.2 - Neugebauer
## Part I - Reading the CSVs, Simple Streaming

In [2]:
stream_dir = '/FileStore/tables/baby-data'
static = spark.read.csv(stream_dir, header=True) 
dataSchema = static.schema 

In [3]:
print(dataSchema)

StructType(List(StructField(state,StringType,true),StructField(sex,StringType,true),StructField(year,StringType,true),StructField(name,StringType,true),StructField(count,StringType,true)))

In [4]:
baby_stream_df = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).csv(stream_dir)

In [5]:
gender_count = baby_stream_df.groupBy('sex').count()

## Start the Stream

In [7]:
gender_count_query = gender_count.writeStream.queryName('gender_counts').format('memory').outputMode('complete').start()

## Periodically Read
This version uses a loop with a delay to read the aggregated DataFrame, which in turn is based on new CSV files being added.

In [9]:
# First part
import time
import datetime

for x in range(10):
  st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
  print("Female count number", x, "at", st)
  spark.sql("SELECT * FROM gender_counts WHERE sex='F'").show()
  time.sleep(10)

Female count number 0 at 2019-10-19 14:19:50
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 1 at 2019-10-19 14:20:00
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 2 at 2019-10-19 14:20:10
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 3 at 2019-10-19 14:20:22
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 4 at 2019-10-19 14:20:32
+---+------+
sex| count|
+---+------+
 F|100361|
+---+------+

Female count number 5 at 2019-10-19 14:20:42
+---+------+
sex| count|
+---+------+
 F|100361|
+---+------+

Female count number 6 at 2019-10-19 14:20:53
+---+------+
sex| count|
+---+------+
 F|133784|
+---+------+

Female count number 7 at 2019-10-19 14:21:05
+---+------+
sex| count|
+---+------+
 F|167264|
+---+------+

Female count number 8 at 2019-10-19 14:21:17
+---+------+
sex| count|
+---+------+
 F|200789|
+---+------+

Female count number 9 at 2019-10-19 14:21:28
+---+------+
sex| count|
+---+------+
 F|234184|
+---+------+

## Part II - Triggers
This version is almost identical to the prior, except a trigger is used.

In [11]:
gender_count_query = gender_count.writeStream.queryName('gender_counts').format("memory").trigger(processingTime='30 seconds').outputMode('complete').start()

In [12]:
# Second part
from time import sleep
import datetime

for x in range(30):
  st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
  print("Female count number", x, "at", st)
  spark.sql("SELECT * FROM gender_counts WHERE sex='F'").show()
  sleep(10)

Female count number 0 at 2019-10-19 14:27:55
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 1 at 2019-10-19 14:28:05
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 2 at 2019-10-19 14:28:15
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 3 at 2019-10-19 14:28:25
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 4 at 2019-10-19 14:28:35
+---+-----+
sex|count|
+---+-----+
 F|33417|
+---+-----+

Female count number 5 at 2019-10-19 14:28:50
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 6 at 2019-10-19 14:29:00
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 7 at 2019-10-19 14:29:10
+---+-----+
sex|count|
+---+-----+
 F|67008|
+---+-----+

Female count number 8 at 2019-10-19 14:29:21
+---+------+
sex| count|
+---+------+
 F|100361|
+---+------+

Female count number 9 at 2019-10-19 14:29:31
+---+------+
sex| count|
+---+------+
 F|100361|
+---+------+

Female count number 10 at 2019-10-19 14:29:41
+---+------+
sex| count|
+---+------+
 F|133784|
+---+------+

Female count number 11 at 2019-10-19 14:29:51
+---+------+
sex| count|
+---+------+
 F|133784|
+---+------+

Female count number 12 at 2019-10-19 14:30:01
+---+------+
sex| count|
+---+------+
 F|133784|
+---+------+

Female count number 13 at 2019-10-19 14:30:12
+---+------+
sex| count|
+---+------+
 F|133784|
+---+------+

Female count number 14 at 2019-10-19 14:30:22
+---+------+
sex| count|
+---+------+
 F|167264|
+---+------+

Female count number 15 at 2019-10-19 14:30:32
+---+------+
sex| count|
+---+------+
 F|167264|
+---+------+

Female count number 16 at 2019-10-19 14:30:51
+---+------+
sex| count|
+---+------+
 F|200789|
+---+------+

Female count number 17 at 2019-10-19 14:31:01
+---+------+
sex| count|
+---+------+
 F|200789|
+---+------+

Female count number 18 at 2019-10-19 14:31:20
+---+------+
sex| count|
+---+------+
 F|234184|
+---+------+

Female count number 19 at 2019-10-19 14:31:30
+---+------+
sex| count|
+---+------+
 F|234184|
+---+------+

Female count number 20 at 2019-10-19 14:31:40
+---+------+
sex| count|
+---+------+
 F|234184|
+---+------+

Female count number 21 at 2019-10-19 14:31:51
+---+------+
sex| count|
+---+------+
 F|267752|
+---+------+

Female count number 22 at 2019-10-19 14:32:01
+---+------+
sex| count|
+---+------+
 F|267752|
+---+------+

Female count number 23 at 2019-10-19 14:32:11
+---+------+
sex| count|
+---+------+
 F|301272|
+---+------+

Female count number 24 at 2019-10-19 14:32:21
+---+------+
sex| count|
+---+------+
 F|301272|
+---+------+

Female count number 25 at 2019-10-19 14:32:31
+---+------+
sex| count|
+---+------+
 F|301272|
+---+------+

Female count number 26 at 2019-10-19 14:32:41
+---+------+
sex| count|
+---+------+
 F|301272|
+---+------+

Female count number 27 at 2019-10-19 14:32:51
+---+------+
sex| count|
+---+------+
 F|334723|
+---+------+

Female count number 28 at 2019-10-19 14:33:01
+---+------+
sex| count|
+---+------+
 F|334723|
+---+------+

Female count number 29 at 2019-10-19 14:33:11
+---+------+
sex| count|
+---+------+
 F|334723|
+---+------+

## Conclusions
What's clear is that the triggers afford a bit more flexibility in terms of time. With the first iteration, I missed quite a bit of data because of timing; the 10th time through, the count was only 234k. Looking at the trigger, the stream waited for data and only added it at the trigger point.